In [ ]:
import xml.etree.ElementTree as ET
import os, os.path
import pickle
import numpy as np

## you are currently overriding entries on the same protein in different organisms! This is occuring around 10 times

In [ ]:
file_directory = os.listdir(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\raw\ASD\ASD_Release_062015_XF')
file_path = r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\raw\ASD\ASD_Release_062015_XF'

paths = []
for filename in file_directory:
    paths.append(os.path.join(file_path,filename))

In [ ]:
ASD_entries = {}
missing = []
ASD_ID_list_with_duplicates = []
for ASD_file in paths:

    try:
        root = ET.parse(ASD_file).getroot()
        ASD_ID = root.find('Organism_ID').text
        Protein_ID = root.find('UniProt_ID').text
        Organism = root.find('Organism').text
        Gene = root.find('Gene')[0].text
        Gene_ID = root.find('Gene')[2].text
        
        modulator_list = []

        try:
            for modulators in root.find('Modulator_List'):
                modulator_list.append((modulators[0].text, modulators[1].text))    
        except:
            pass

        ASD_entries[ASD_ID] = {
            'Protein_ID' : Protein_ID,
            'Organism' : Organism,
            'Gene' : Gene,
            'Gene_ID' : Gene_ID,
            'Modulators' : modulator_list
        }

        ASD_ID_list_with_duplicates.append((ASD_ID))
        
        
    except:
        missing.append(ASD_file)

print(len(missing))


In [ ]:
# this code generates logcal tripples with IDs 

ASD_ID_list = list(dict.fromkeys(ASD_ID_list_with_duplicates))

triples_with_IDs = []
for protein_ASD in ASD_ID_list:
    interactions = ASD_entries[protein_ASD]['Modulators']
    for allo_ASD in interactions:
        triples_with_IDs.append([allo_ASD[0], allo_ASD[1], protein_ASD])


In [ ]:
#This generates hash tables for each of the things in the tripples 

objects = list(set([item[0] for item in triples_with_IDs]))
relationships = list(set([item[1] for item in triples_with_IDs]))
subjects = list(set([item[2] for item in triples_with_IDs]))

subject_map = {}
subject_lookup = {}
for subject_index, subject in enumerate(subjects):
    subject_map[subject] = subject_index
    subject_map[subject_index] = subject

relationship_map = {}
relationship_lookup = {}
for relationship_index, relationship in enumerate(relationships):
    relationship_map[relationship] = relationship_index+1
    relationship_map[relationship_index+1] = relationship

object_map = {}
object_lookup = {}
for object_index, obj in enumerate(objects):
    object_map[obj] = object_index
    object_lookup[object_index] = obj


nodes= list(set([item[0] for item in triples_with_IDs] + [item[2] for item in triples_with_IDs]))
node_map = {}
for node_index, node in enumerate(nodes):
    node_map[node] = node_index
    node_map[node_index] = node
    max_node = node_index



In [ ]:
categorical_triplets = []
for triplet in triples_with_IDs:
    categorical_triplets.append([
        object_map[triplet[0]],
        relationship_map[triplet[1]],
        subject_map[triplet[2]]
    ])

same_space_triplets = []
for triplet in triples_with_IDs:
    same_space_triplets.append([
        node_map[triplet[0]],
        relationship_map[triplet[1]],
        node_map[triplet[2]]
    ])

In [ ]:
from scipy.sparse import csr_matrix
import numpy as np

Output = {}
for x, y, z in same_space_triplets:
    if (x,y) in Output:
        Output[(x,y)].append(z)
    else:
        Output[(x,y)] = [z]

rows = []
cols = []
data = []

for index, (x, y, z) in enumerate(same_space_triplets):
    vector = Output[(x,y)]
    v_len = len(vector)
    r = np.full(v_len, index, dtype=int)
    c = np.array(vector)
    d = np.ones(v_len)
    rows.append(r)
    cols.append(c)
    data.append(d)

rows = np.hstack(rows)
cols = np.hstack(cols)
data = np.hstack(data)
no_nodes = int(len(node_map)/2)
no_rows = len(same_space_triplets)

sparse_data = csr_matrix((data, (rows, cols)), shape=(no_rows, no_nodes))

In [ ]:
BCE_dictionary = {}
for index, (o, r, s)  in enumerate(same_space_triplets):
    if (o, r) in BCE_dictionary:
        pass
    else:
        BCE_dictionary[(o, r)] = sparse_data[index]

In [ ]:
# this code generates an edge-list graph, if ever required

nodes= list(set([item[0] for item in triples_with_IDs] + [item[2] for item in triples_with_IDs]))

node_hash_map = {}
for node_index, node in enumerate(nodes):
    node_hash_map[node] = node_index

edge_list = []
edge_features = []
for protein in ASD_ID_list:
    interactions = ASD_entries[protein]['Modulators']
    for ASD in interactions:
        edge_list.append([
            node_hash_map[protein],
            node_hash_map[ASD[0]]])
        edge_features.append(ASD[1])


In [ ]:
from sklearn.preprocessing import OneHotEncoder

edge_features2 = [[x] for x in edge_features] 

encoder = OneHotEncoder()
encoder.fit(edge_features2)

one_hots = encoder.transform(edge_features2).toarray().tolist()

In [ ]:
print(triples_with_IDs[0])

print(same_space_triplets[0])

print(categorical_triplets[0])

print(edge_list[0])

print(edge_features[0])

print(one_hots[0])

In [ ]:
with open('edge_list.pickle', 'wb') as handle:
    pickle.dump((edge_list), handle)
with open('edge_features.pickle', 'wb') as handle:
    pickle.dump((one_hots), handle)

In [ ]:
with open('triplets.pickle', 'wb') as handle:
    pickle.dump(np.array(same_space_triplets), handle)

with open('triplet_lookup.pickle', 'wb') as handle:
    pickle.dump(node_map, handle)

with open('ASD_dictionary.pickle', 'wb') as handle:
    pickle.dump(ASD_entries, handle)

with open('Sparse_dictionary.pickle', 'wb') as handle:
    pickle.dump(BCE_dictionary, handle)